In [37]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.optimize import minimize,broyden1

## short rate lattice

In [2]:
def short_rate_lattice(r_i=0.05,u=1.1,d=0.9,n=10):
    """creates a short rate lattice given the initial short rate,up factor, down factor and number of periods
    
    Parameters:
    r_i(float): the initial short rate
    u(float): up-factor
    d(float): down-factor
    n(int): no of periods
    
    Returns:
    np.ndarray: A 2-D numpy array with the short rate lattice"""
    
    rates=np.zeros((n+1,n+1))
    rates[0][0]=r_i
    
    for i in range(1,n+1):
        rates[i][0]=rates[i-1][0]*d
        
        for j in range(1,i+1):
            rates[i][j]=rates[i-1][j-1]*u
            
    return rates

short_rates=short_rate_lattice().round(3)

## pricing of zero coupon bonds
### without coupon bearing

In [8]:
def zcb(r_i=0.05,u=1.1,d=0.9,coupon=100.0,del_t=10,q=0.5):
    """creates a zero coupon bond lattice with initial short rate, up factor, down factor, coupon value
    
    Parameters:
    r_i(float): the initial short rate
    u(float): up-factor
    d(float): down-factor
    coupon(float): value upon maturity
    n(int): no of periods
    q(float): probability of up factor
    Returns:
    np.ndarray: A 2-D numpy array with the coupon price lattice
    """
    rates=short_rate_lattice(r_i,u,d,del_t)
    prices=np.zeros((del_t+1,del_t+1))
    for j in range(del_t+1):
        prices[del_t][j]=coupon
    for i in range(del_t-1,-1,-1):
        for j in range(i+1):
            prices[i][j]=(q*prices[i+1][j]+(1-q)*prices[i+1][j+1])/(1+rates[i][j])
    return prices.round(1)
sr=short_rate_lattice()

In [9]:
a=zcb()

### with coupon bearing  

In [10]:
def coupon_bearing_bond(r_i=0.05,u=1.1,d=0.9,coupon=100,n=10,q=0.5,del_t=4,per=0.1,print_matrice=False):
    
    """returns the forward price of the coupon bearing bond with initial short rate, up factor,
       down factor, coupon value, delivery time, divident percent return and the boolean expression of
       printing forward prices matrice
    
    Parameters:
    r_i(float): the initial short rate
    u(float): up-factor
    d(float): down-factor
    coupon(float): value upon maturity
    n(int): no of periods
    q(float): probability of up factor
    del_t(int): delivery time of bond dividends
    per(float): dividend percent return
    print_matrice(boolean): prints the coupon bearing bond lattice if set to true
    int: forward price of the coupon bearing bond
    Returns:
    np.ndarray: A 2-D numpy array with the coupon price lattice"""
    
    rates=short_rate_lattice(r_i,u,d,n)
    prices=np.zeros((n+1,n+1))
    for j in range(n+1):
        prices[n][j]=(1+per)*coupon
        for i in range(n-1,-1,-1):
            for j in range(i+1):
                prices[i][j]=(q*prices[i+1][j]+(1-q)*prices[i+1][j+1])/(1+rates[i][j])
                if i==del_t+1: prices[i][j]+=coupon*per
    if print_matrice:
        return prices.round(2)
    
    a=zcb(r_i,u,d,coupon,del_t,q)[0][0]
    return (coupon*prices[0][0]/a).round(2)

In [11]:
coupon_bearing_bond(print_matrice=True)


array([[ 75.62,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [ 82.57,  76.24,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [ 89.05,  83.53,  77.33,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [ 94.98,  90.32,  85.  ,  79.01,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [100.35,  96.55,  92.15,  87.11,  81.43,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [105.12, 102.16,  98.68,  94.65,  90.02,  84.77,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ],
       [ 99.05,  96.81,  94.15,  91.03,  87.39,  83.19,  78.39,   0.  ,
          0.  ,   0.  ,   0.  ],
       [102.47, 100.9 ,  99.01,  96.77,  94.12,  91.02,  87.41,  83.25,
          0.  ,   0.  ,   0.  ],
       [105.41, 104.43, 103.25, 101.84, 100.15,  98.14,  95.77,  92.98,
         89.74,   0.  ,   0.  ],
       [107.91, 107.46, 106.91, 106.24, 105.44, 104.48,

### pricing of European options on zero coupon bonds(ZCB)

In [12]:
def european_op(zcb=a,exp_t=2,K=84.,sr_lat=sr,q=0.5,type_="c"):
    """
    computes european option price of a zcb with no coupon bearing with expiration time and strike price
    
    Parameters:
    zcb(2-D numpy array): zero coupon bond pricing lattice
    exp_t(int): expiration time
    K(float): strike price
    q(float): probability of up factor
    type_(char): type of option, default is call(c), other option is put(p)
    Returns:
    np.ndarray: A 2-D numpy array with the coupon price lattice
    """
    
    zcb=zcb[:exp_t+1]
    n=zcb.shape[0]
    new=np.zeros((n,n))
    payoff=zcb[-1:][0]
    
    if type_=="c":
        for i in range(n):
            payoff[i]=max(payoff[i]-K,0)
            new[n-1][i]=payoff[i]
        
        for i in range(exp_t-1,-1,-1):
            for j in range(i+1):
                new[i][j]=(q*new[i+1][j]+(1-q)*new[i+1][j+1])/(1+sr[i][j])
    
    elif type_=="p":
        for i in range(n):
            payoff[i]=max(K-payoff[i],0)
            new[n-1][i]=payoff[i]
        
        for i in range(exp_t-1,-1,-1):
            for j in range(i+1):
                new[i][j]=(q*new[i+1][j]+(1-q)*new[i+1][j+1])/(1+sr[i][j])
    else:
        print("Invalid type, returning empty matrice")
    return new

In [13]:
a=zcb()
european_op(type_='p')

array([[14.54713907,  0.        ,  0.        ],
       [12.91866029, 17.63033175,  0.        ],
       [11.1       , 15.9       , 21.3       ]])

In [14]:
def american_op(zcb=a,exp_t=6,K=80.,sr_lat=sr,q=0.5,type_="c"):
    """computes American put option price of a zcb with no coupon bearing with expiration time and strike price
    
    Parameters:
    zcb(2-D numpy array): zero coupon bond pricing lattice
    exp_t(int): expiration time
    K(float): strike price
    q(float): probability of up factor
    type_(char): type of option, default is put(p), other option is call(c)"""
    zcb=zcb[:exp_t+1]
    n=zcb.shape[0]
    new=np.zeros((n,n))
    payoff=zcb[-1:][0]
    if type_=="p":
        for i in range(n):
            payoff[i]=max(K-payoff[i],0)
            new[n-1][i]=payoff[i]
    
        for i in range(exp_t-1,-1,-1):
            for j in range(i+1):
                new[i][j]=max(K-zcb[i][j],(q*new[i+1][j]+(1-q)*new[i+1][j+1])/(1+sr[i][j]))
                
    elif type_=="c":
        for i in range(n):
            payoff[i]=max(payoff[i]-K,0)
            new[n-1][i]=payoff[i]
    
        for i in range(exp_t-1,-1,-1):
            for j in range(i+1):
                new[i][j]=max(zcb[i][j]-K,(q*new[i+1][j]+(1-q)*new[i+1][j+1])/(1+sr[i][j]))
    else:
        print("Invalid type, returning empty matrice")
    return new

In [15]:
a=zcb()
american_op(type_="c").round(2)

array([[ 2.37,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 3.4 ,  1.57,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 4.65,  2.46,  0.85,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 6.03,  3.66,  1.51,  0.29,  0.  ,  0.  ,  0.  ],
       [ 7.41,  5.09,  2.55,  0.63,  0.  ,  0.  ,  0.  ],
       [ 8.74,  6.56,  4.02,  1.33,  0.  ,  0.  ,  0.  ],
       [10.  ,  8.  ,  5.6 ,  2.8 ,  0.  ,  0.  ,  0.  ]])

### futures contract of bonds pricing

In [16]:
def future_cb(cbb=a,exp_t=4,q=.5,print_matrice=False):
    """
    Returns np.darray if print_matrice is set to true, else returns the bond futures price at initial time"""
    ca = cbb[exp_t]
    prices=np.zeros((exp_t+1,exp_t+1))
    for j in range(exp_t+1):
        prices[exp_t][j]=ca[j]

    for i in range(exp_t-1,-1,-1):
        for j in range(i+1):
            prices[i][j]=(q*prices[i+1][j]+(1-q)*prices[i+1][j+1])
            
    if print_matrice:
        return prices
    
    return prices[0][0]

In [17]:
future_cb(print_matrice=True,exp_t=4)
short_rates

array([[0.05 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.045, 0.055, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.041, 0.05 , 0.061, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.036, 0.045, 0.054, 0.067, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.033, 0.04 , 0.049, 0.06 , 0.073, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.03 , 0.036, 0.044, 0.054, 0.066, 0.081, 0.   , 0.   , 0.   ,
        0.   , 0.   ],
       [0.027, 0.032, 0.04 , 0.049, 0.059, 0.072, 0.089, 0.   , 0.   ,
        0.   , 0.   ],
       [0.024, 0.029, 0.036, 0.044, 0.053, 0.065, 0.08 , 0.097, 0.   ,
        0.   , 0.   ],
       [0.022, 0.026, 0.032, 0.039, 0.048, 0.059, 0.072, 0.088, 0.107,
        0.   , 0.   ],
       [0.019, 0.024, 0.029, 0.035, 0.043, 0.053, 0.065, 0.079, 0.096,
        0.118, 0.   ],
       [0.017, 0.021, 0.026, 0.032, 0.039, 0.048, 0.058, 0.0

### caplet pricing

In [21]:
def caplet(K=0.02,t=6,sr_lattice=q_i,p=.5,print_matrice=True):
    """
    """
    q=sr_lattice[-1:][0]
    arr=np.zeros((t,t))
    
    for j in range(t):
        arr[t-1][j]= max(0,q[j]-K)
        arr[t-1][j]=arr[t-1][j]/(1+q_i[t-1][j])
    
    for i in range(t-2,-1,-1):
        for j in range(i+1):
            arr[i][j]=(p*arr[i+1][j]+(1-p)*arr[i+1][j+1])/(1+q_i[i][j])
    if print_matrice:
        return arr.round(4)
    return arr[0][0]

In [22]:
q_i=short_rate_lattice(r_i=0.06,u=1.25,d=0.9,n=5).round(3)
caplet()

array([[0.042 , 0.    , 0.    , 0.    , 0.    , 0.    ],
       [0.0375, 0.0515, 0.    , 0.    , 0.    , 0.    ],
       [0.0321, 0.047 , 0.0637, 0.    , 0.    , 0.    ],
       [0.0262, 0.0411, 0.0593, 0.0801, 0.    , 0.    ],
       [0.0203, 0.0344, 0.0527, 0.0758, 0.1033, 0.    ],
       [0.0145, 0.0276, 0.0449, 0.0685, 0.0989, 0.1378]])

### pricing swaps

In [23]:
def swaps(K=0.05,t=6,first_t=1,sr_lattice=q_i,p=.5,print_matrice=False):
    """
    """
    q=q_i[-1:][0]
    arr=np.zeros((t,t))
    
    for j in range(t):
        arr[t-1][j]= q[j]-K
        arr[t-1][j]=arr[t-1][j]/(1+q_i[t-1][j])
    for i in range(t-2,-1,-1):
        for j in range(i+1):
            arr[i][j]=(p*arr[i+1][j]+(1-p)*arr[i+1][j+1])/(1+q_i[i][j])
            if i>=first_t-1:
                arr[i,j]+=(q_i[i][j]-K)/(1 + q_i[i, j])
    if print_matrice:
        return arr.round(5)
    return arr[0][0]

In [24]:
swaps(print_matrice=True)

array([[ 0.0992 ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.05001,  0.1403 ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.01417,  0.08325,  0.16839,  0.     ,  0.     ,  0.     ],
       [-0.0085 ,  0.04023,  0.10159,  0.17886,  0.     ,  0.     ],
       [-0.01802,  0.01228,  0.05109,  0.10115,  0.16443,  0.     ],
       [-0.01449, -0.00095,  0.01685,  0.0411 ,  0.07244,  0.11243]])

### swaption pricing

In [25]:
def swaption(K=0.05,exp_t=3,first_t=1,sr_lattice=q_i,p=.5,print_matrice=True):
    """ pricing of european call swaptions
    """
    t=q_i.shape[0]
    q=q_i[-1:][0]
    prices=swaps(K,t,first_t,sr_lattice,p,print_matrice=True)
    arr=prices
    for j in range(t):
        arr[exp_t][j]=max(0,arr[exp_t,j])
        
    for i in range(exp_t-1,-1,-1):
        for j in range(i+1):
            arr[i][j]=(p*arr[i+1][j]+(1-p)*arr[i+1][j+1])/(1+q_i[i][j])
    
    if print_matrice:
        return arr.round(5)
    return arr[0][0]
swaption()

array([[ 0.06184,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.04059,  0.0905 ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.01918,  0.0664 ,  0.12818,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.04023,  0.10159,  0.17886,  0.     ,  0.     ],
       [-0.01802,  0.01228,  0.05109,  0.10115,  0.16443,  0.     ],
       [-0.01449, -0.00095,  0.01685,  0.0411 ,  0.07244,  0.11243]])

### Forward equations

In [27]:
def forward_eqn_lattice(short_rate_lattice=q_i,for_spot_rate=False):
    """computes and returns forward equation lattice"""
    n=short_rate_lattice.shape[0]
    fwd=np.zeros((n+1,n+1))
    fwd[0,0]=1.
    fwd[1,0]=1/(2*(1+short_rate_lattice[0,0]))
    fwd[1,1]=1/(2*(1+short_rate_lattice[0,0]))
    
    for i in range(2,n):
        fwd[i][0]= 0.5*fwd[i-1,0]/(1+short_rate_lattice[i-1,0])
        for j in range(1,i+1):
            fwd[i][j]= ((fwd[i-1,j]/(2*(1+short_rate_lattice[i-1,j]))) + (fwd[i-1,j-1]/(2*(1+short_rate_lattice[i-1,j-1]))))
    
    fwd[n][0]=0.5*fwd[n-1,0]/(1+short_rate_lattice[n-1,0])
    fwd[n][n]=0.5*fwd[n-1,n-1]/(1+short_rate_lattice[n-1,n-1])
    
    for j in range(1,n):
        fwd[n][j]=((fwd[n-1,j]/(2*(1+short_rate_lattice[n-1,j]))) + (fwd[n-1,j-1]/(2*(1+short_rate_lattice[n-1,j-1]))))
        
    if for_spot_rate==False:
        fwd=np.delete(fwd,n,0)
        fwd=np.delete(fwd,n,1)
    return fwd.round(8)
        
forward_eqn_lattice()    

array([[1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.47169811, 0.47169811, 0.        , 0.        , 0.        ,
        0.        ],
       [0.22376571, 0.44316018, 0.21939447, 0.        , 0.        ,
        0.        ],
       [0.10665668, 0.31412867, 0.30774369, 0.1002717 , 0.        ,
        0.        ],
       [0.05108078, 0.19911503, 0.28998244, 0.18683257, 0.04488438,
        0.        ],
       [0.02458171, 0.11894902, 0.22911752, 0.21928983, 0.10412269,
        0.01958306]])

### Black Derman Toy (BDT) model

In [28]:
def BDT(n=14,a=14*[5.],b=0.005):
    """"""
    lattice=np.zeros((n,n))
    for i in range(n):
        for j in range(i+1):
            lattice[i,j]=a[i]*np.exp(b*j)
    return lattice.round(2)/100
q_i=BDT(n=14,a=14*[5.],b=0.005)

### building forward rate lattice using BDT lattice

In [95]:
def forward_price_lattice(short_rate_lattice=BDT()):
    zcb_bdt=forward_eqn_lattice(short_rate_lattice,for_spot_rate=True).round(3)
    zcb_bdt_p=np.zeros(zcb_bdt.shape[0]-1)
    for i in range(zcb_bdt.shape[0]-1):
        zcb_bdt_p[i]=zcb_bdt[i+1].sum()
    return zcb_bdt_p.round(3)


#### calculating the spot rates

In [96]:
def spot_rate(price_arr=zcb_bdt_p):
    n=len(price_arr)
    arr=np.zeros(n)
    for i in range(n):
        arr[i]=(1/price_arr[i])**(1/(i+1))
    return arr-1

### calibration of BDT model

#### mse()


In [116]:
def mse(actual,predicted):
    return ((actual-predicted)**2).sum()
# rates=[3.0,3.1,3.2,3.3, 3.4,3.5,3.55,3.6,3.65,3.7]
rates=[7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22,11.55,11.92,12.2,12.32]

### manual calibration

In [42]:
# a=np.linspace(5,8,10)
# b=np.linspace(0.00001,0.25,1000)
# arr=[]
# val_b=[]
# val_a=[]
# n=14
# for i in range(len(a)):
#     for j in range(len(b)):
#         q=BDT(a=n*[a[i]],b=b[j])
#         forward_price_lattice(q)
#         z=spot_rate(forward_price_lattice(q))
#         arr.append(mse(100*z,rates))
#         val_b.append(b[j])
#         val_a.append(a[i])

In [43]:
# print("min loss: ",min(arr))
# print("min loss value of arr: ",arr.index(min(arr)))
# print("min loss value of b: ",val_b[arr.index(min(arr))])
# print("min loss value of a: ",val_a[arr.index(min(arr))])
      
# # arr.index(min(arr))

### calibrating the BDT model using scipy
- objective: minimize mse between 100*z and rates
- constraint1: z between 0 and 1

In [153]:
actual_spot_rates = np.array([7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22,11.55,11.92,12.2,12.32])
actual_spot_rates=actual_spot_rates/100
# Define objective function to minimize
def objective_function(a):
    # Calculate predicted spot rates using given values of a
    short_rate_lattice = BDT(a=a)
    forward_prices = forward_price_lattice(short_rate_lattice)
    predicted_spot_rates = spot_rate(forward_prices)

    # Calculate mean squared error
    mse = np.sum((actual_spot_rates - predicted_spot_rates)**2) / len(actual_spot_rates)

    return mse

a0 = np.array([5.0] * 14)
b=(0.0,100.0)
bnds=(b,)*14
# Minimize objective function to find optimal values of a
result = minimize(objective_function, a0, method='Powell',bounds=bnds)

# Extract optimal values of a
optimal_a = result.x
result
# # Print optimal values of a
# print("Optimal values of a:")
# for i in range(len(optimal_a)):
#     print("Time {}: {:.2f}".format(i+1, optimal_a[i]))

   direc: array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.0000000

In [156]:
result.x
spot_rate(forward_price_lattice(BDT(a=result.x)))*100

array([ 7.29613734,  7.58287073,  8.08321954,  8.55926041,  9.233992  ,
        9.69361555, 10.03551051, 10.4964781 , 10.74811549, 11.22901479,
       11.53291317, 11.87993902, 12.23553864, 12.30401151])

### swaption pricing in BDT model
- Consider a 2−8 payer swaption in the BDT model with fixed rate = 11.65%
- an option to enter an 8-year swap in 2 years time
- settled in arrears so payments would take place in years 3 through 10
- each payment based on prevailing short-rate of previous year
- "payer" feature of option means that if option is exercised, the exerciser “pays fixed and receives floating"
- owner of a receiver swaption would “receive fixed and pay floating.

In [205]:
## calibrated rates(a_i) for b=0.005
rates=[7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22,11.55,11.92,12.2,12.32]

x0=np.array(14*[5.]) ## initialising the values of a
b=(0.0,100.0)
bnds=(b,)*14
q=BDT(n=14,a=x0,b=0.005)
forward_price_lattice(q)
z=spot_rate(forward_price_lattice(q))

soln=minimize(objective_function,x0,method="Powell",bounds=bnds)
q_i=BDT(n=10,a=soln.x)

spot_rate(forward_price_lattice(q_i))

1e6*swaption(K=0.1165,exp_t=8,first_t=1,sr_lattice=q_i)

array([[15940.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.],
       [16860., 17330.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.],
       [17930., 18450., 18970.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.],
       [19260., 19840., 20420., 20990.,     0.,     0.,     0.,     0.,
            0.,     0.],
       [20870., 21510., 22160., 22800., 23450.,     0.,     0.,     0.,
            0.,     0.],
       [22950., 23680., 24420., 25150., 25890., 26620.,     0.,     0.,
            0.,     0.],
       [25280., 26120., 26960., 27790., 28640., 29480., 30320.,     0.,
            0.,     0.],
       [27840., 28800., 29770., 30700., 31670., 32640., 33600., 34550.,
            0.,     0.],
       [31000., 32050., 33210., 34290., 35370., 36530., 37600., 38760.,
        39820.,     0.],
       [26080., 26680., 27270., 27950., 28540., 29210., 29810., 30480.,
        31070., 31740.]])

### swaption pricing in BDT model
- Once your model has been calibrated, compute the price of a payer swaption with notional 
- $1M that expires at time t=3
- option strike of 00. 
- fixed rate of 3.9 and that if the option is exercised then cash-flows take place at times 

In [209]:
x0=np.array(14*[5.]) ## initialising the values of a
b=(0.0,100.0)
bnds=(b,)*14
q=BDT(n=14,a=x0,b=0.005)
forward_price_lattice(q)
z=spot_rate(forward_price_lattice(q))

soln=minimize(objective_function,x0,method="Powell",bounds=bnds)
q_i=BDT(n=14,a=soln.x)

spot_rate(forward_price_lattice(q_i))

1e6*swaption(K=0.039,exp_t=3,first_t=1,sr_lattice=q_i)

array([[412730.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [441880., 443420.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [475900., 477760., 479570.,      0.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [517900., 520130., 522350., 524560.,      0.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [507530., 509750., 511960., 514160., 516350.,      0.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [487670., 489860., 492040., 494210., 496380., 498540.,      0.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],
       [464120., 466260., 468420., 470540., 472660., 474800., 476920.,
             0.,      0.,      0.,      0.,      0.,      0.,      0.],

In [210]:
q_i=BDT(n=10,a=soln.x)
spot_rate(forward_price_lattice(q_i))
1e6*swaption(K=0.039,first_t=3,exp_t=9,sr_lattice=q_i)
q_i

array([[0.0725, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.0791, 0.0795, 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.0906, 0.091 , 0.0915, 0.    , 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.1001, 0.1006, 0.1011, 0.1016, 0.    , 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.1173, 0.1179, 0.1185, 0.1191, 0.1197, 0.    , 0.    , 0.    ,
        0.    , 0.    ],
       [0.12  , 0.1206, 0.1212, 0.1218, 0.1224, 0.123 , 0.    , 0.    ,
        0.    , 0.    ],
       [0.1204, 0.121 , 0.1216, 0.1222, 0.1228, 0.1234, 0.124 , 0.    ,
        0.    , 0.    ],
       [0.1324, 0.133 , 0.1337, 0.1344, 0.135 , 0.1357, 0.1364, 0.1371,
        0.    , 0.    ],
       [0.125 , 0.1256, 0.1263, 0.1269, 0.1275, 0.1282, 0.1288, 0.1295,
        0.1301, 0.    ],
       [0.1464, 0.1471, 0.1478, 0.1486, 0.1493, 0.1501, 0.1508, 0.1516,
        0.1523, 0.1531]])

## defaultable bond pricing

In [784]:
hazard_rates=0.02
def defaultable_bond(F=100,n=4,c=0.02,interest_rate=0.025,R=0.25,hazard_rate=0.02,return_h_rate=False):
    """returns the price of a defaultable bond 
    Parameters:
    F: Face value of the bond (int)
    n: time period of the bond(time to maturity) (int)
    interest_rate: interest rate of the market at initial period (float)
    R: recovery rate (float)
    return_h_rate: returns the hazard rate array if set to True, Used for calibration of model (boolean)
    Returns:
    price of the bond (int)"""
    hazard_rates=n*[hazard_rate]
    
    cpf=n*[c*F]
    cpf[n-1]+=F

    prob_s=np.zeros(n+1)
    prob_d=np.zeros(n+1)
    discount_rate=np.zeros(n+1)
    prob_s[0]=1
    prob_d[0]=0
    discount_rate[0]=1
    for i in range(1,n+1):
        prob_s[i]=prob_s[i-1]*(1-hazard_rates[i-1])
        prob_d[i]=prob_s[i-1]*hazard_rates[i-1]
        discount_rate[i]=1/(1 + interest_rate)**i
    
    dr=discount_rate[1:]
    p_s=prob_s[1:]
    p_d=prob_d[1:]
    a=dr*(p_s*cpf+p_d*F*R)
    
    if return_h_rate:
        return hazard_rates, a.sum()
    
    return a.sum()

In [786]:
a=defaultable_bond(c=0.05,n=2,R=0.1,return_h_rate=True)
a

([0.02, 0.02], 101.14503271861986)

### calibrating the model by tuning the hazard rates

- objective : minimize the mse
- constraint : h between 0 and 1

In [ ]:
true_price=[100.92,91.56,105.60,98.90,137.48]
predicted=[]
predicted.append(defaultable_bond(c=0.05,R=0.1,n=2)

### Credit Default Swaps(CDS) pricing 
#### Setting of the perfect float rate that is risk neutral and arbitrage free for both seller and buyer

In [983]:
principal=1000000
hazard_rate=0.01
n=2
R=0.45
theta=4 ## no of compounds in a year
interest=0.01 ## quarterly(in this case interest rates)
interest_rate=interest/theta
n=n*theta
spread=160 ## in bps

hazard_rates=n*[hazard_rate]
prob_s=np.zeros(n+1)
prob_d=np.zeros(n+1)
discount_rate=np.zeros(n+1)
premium=np.zeros(n+1)
acc_interest=np.zeros(n+1)


prob_s[0]=1
prob_d[0]=0
discount_rate[0]=1
acc_interest[0]=0

for i in range(1,n+1):
    prob_s[i]=prob_s[i-1]*(1-hazard_rates[i-1])
    prob_d[i]=prob_s[i-1]*hazard_rates[i-1]
    discount_rate[i]=1/(1 + interest_rate)**i
    premium[i]=(principal*spread)*(discount_rate[i]*prob_s[i])/(theta*10000)
acc_interest=premium*prob_d*0.5

In [985]:
acc_interest

array([ 0.        , 19.75062344, 19.30931275, 18.87786277, 18.45605317,
       18.04366854, 17.64049829, 17.24633653, 16.86098198])

In [988]:
principal*spread*0.25*1e-4

4000.0

In [986]:
prob_d

array([0.        , 0.01      , 0.0099    , 0.009801  , 0.00970299,
       0.00960596, 0.0095099 , 0.0094148 , 0.00932065])

In [973]:
prob_s

array([1.        , 0.99      , 0.9801    , 0.970299  , 0.96059601,
       0.95099005, 0.94148015, 0.93206535, 0.92274469])